# SVM via Tensorflow

Reference:
* http://www.cnblogs.com/jerrylead/archive/2011/03/18/1988419.html
* https://www.cnblogs.com/vipyoumay/p/7560061.html

## 1. Theory

Given $(X_i, y_i), y_i=+1/-1,$ we try to maximize 
$$ \gamma = \frac{|W^T \cdot X + b|}{||W||}$$
With restriction
$$ 
\begin{cases}
    W^T \cdot X_i + b \geq +1, & \text{if } y_i=1\\ 
    W^T \cdot X_i + b \leq -1, & \text{if } y_i=-1
\end{cases}
$$

### 1.1 Equation Formalization
Using mathmetic refactoration, we get our **SVM algorithm goal formalized** as follow:
$$
\begin{equation}
\max_{W,b} \frac{2}{||W||}, \\
\text{ s.t. } y_i(W^T \cdot X_i + b) \geq 1, i = 1,2,...,m
\end{equation}
$$

Futhermore, $||w|| > 0$, we can equally use:
$$
\begin{equation}
\min_{W,b} \frac{||W||^2}{2}, \\
\text{ s.t. } y_i(W^T \cdot X_i + b) \geq 1, i = 1,2,...,m
\end{equation}
$$


### 1.2 Dual Problem
$$ L(W, b, \alpha) = \frac{1}{2}||W||^2 + \sum_{i=1}^{m} \alpha_i [1-y_i(W^T \cdot X_i + b)], \text{   }\alpha=(\alpha_1;\alpha_2;...;\alpha_m)$$
Make partial devative for W and b, get as follow:
$$ W = \sum_{i=1}^{m} \alpha_i y_i x_i $$
$$ 0 = \sum_{i=1}^{m} \alpha_i y_i $$

Then get into $ L(W, b, \alpha) $, we get SVM dual problem as follow:
$$ \max_{\alpha} \sum_{i=1}^{m} \alpha_i - \frac{1}{2} \sum_{i=1}^m \sum_{j=1}^m \alpha_i \alpha_j y_i y_j x_i^T x_j $$ 
$$ \text{   s.t.} \sum_{i=1}^m \alpha_i y_i = 0, \alpha_i \geq 0, i=1,2,...,m$$

Sovle wtih $\alpha$, get W and b to get model:
$$ f(x) = W^T \cdot X + b = \sum_{i=1}^m \alpha_i y_i x_i^T X + b $$

Satisfy **KKT (Karush-Kuhn-Tucker) condition** as follow:
$$
\begin{cases}
\alpha_i \geq 0; \\
y_i f(x_i) - 1 \geq 0; \\
\alpha_i [y_i f(x_i) - 1] = 0.
\end{cases}
$$